# CONFLUENCE Tutorial: Lumped Basin Workflow (Bow River at Banff)

This notebook walks through the complete CONFLUENCE workflow for a lumped basin model using the Bow River at Banff as an example. We'll execute each step individually to understand what's happening at each stage.

## Introduction to CONFLUENCE

CONFLUENCE (Community Optimization and Numerical Framework for Large-domain Understanding of Environmental Networks and Computational Exploration) is designed to address a fundamental challenge in hydrological modeling: the overwhelming number of decisions required to set up and run a hydrological model.

### The Challenge of Hydrological Modeling Decisions

Hydrological modeling involves numerous interconnected decisions at every stage:
- **Domain Definition**: What area should we model? How should we delineate the watershed?
- **Data Processing**: Which forcing datasets? What temporal and spatial resolution? 
- **Model Selection**: Which hydrological model? What process representations?
- **Parameterization**: Which parameters to calibrate? What are reasonable ranges?
- **Evaluation**: What metrics? Which observation datasets?

Each decision impacts the others, creating a complex decision tree that can be overwhelming for both novice and experienced modelers.

### How CONFLUENCE Helps

CONFLUENCE addresses this complexity by:
1. **Organizing the workflow** into clear, sequential steps
2. **Standardizing data structures** across different models and datasets
3. **Providing sensible defaults** while allowing full customization
4. **Maintaining detailed provenance** of all decisions made
5. **Enabling reproducibility** through comprehensive configuration management

### CONFLUENCE's Code Structure: Organized by Function

CONFLUENCE uses an object-oriented design where different aspects of hydrological modeling are handled by specialized classes called "managers". This is like having different experts, each responsible for their domain:

```python
# Each manager handles a specific task
project_manager = ProjectManager(config, logger)     # Project setup
domain_manager = DomainManager(config, logger)       # Watershed delineation  
data_manager = DataManager(config, logger)           # Data processing
model_manager = ModelManager(config, logger)         # Model operations
```

### Why This Matters

Instead of one giant script with thousands of lines, CONFLUENCE breaks the workflow into logical pieces:

```python
# Traditional approach - everything mixed together
def run_model():
    # 1000+ lines of mixed code...
    setup_directories()
    download_data()
    delineate_watershed()
    process_forcings()
    run_simulation()
    # etc...

# CONFLUENCE approach - organized by function
project_manager.setup_project()          # All project setup code in one place
domain_manager.define_domain()           # All watershed code in one place
data_manager.acquire_forcings()          # All data code in one place
model_manager.run_model()               # All model code in one place
```

This organization makes it easier to:

Find and modify specific functionality
Add new models or data sources
Debug issues in specific components
Reuse code for different projects

Throughout this tutorial, you'll see how each manager handles its part of the workflow, working together to complete the full modeling process.


## Overview of This Tutorial

We'll work through the simplest case in hydrological modeling: a lumped basin model. This treats the entire watershed as a single unit, making it an ideal starting point for understanding the CONFLUENCE workflow.

We'll run through:
1. Project setup and configuration
2. Domain definition (watershed delineation)
3. Data acquisition (forcings and attributes)
4. Model preprocessing
5. Model execution
6. Results visualization

## 1. Setup and Import Libraries

In [ ]:

# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import contextily as cx

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE components
from utils.project.project_manager import ProjectManager
from utils.project.workflow_orchestrator import WorkflowOrchestrator
from utils.config.config_utils import ConfigManager
from utils.config.logging_manager import LoggingManager

# Set up plotting style
plt.style.use('default')
%matplotlib inline

## 2. Initialize CONFLUENCE
First, let's set up our directories and load the configuration. CONFLUENCE uses a centralized configuration file that controls all aspects of the modeling workflow.

In [ ]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Verify paths exist
if not CONFLUENCE_CODE_DIR.exists():
    raise FileNotFoundError(f"CONFLUENCE code directory not found: {CONFLUENCE_CODE_DIR}")

if not CONFLUENCE_DATA_DIR.exists():
    print(f"Data directory doesn't exist. Creating: {CONFLUENCE_DATA_DIR}")
    CONFLUENCE_DATA_DIR.mkdir(parents=True, exist_ok=True)

# Load the template configuration
config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'

# Update the config with our directory paths
config_manager = ConfigManager(config_path)
config = config_manager.config

# Update paths
config['CONFLUENCE_CODE_DIR'] = str(CONFLUENCE_CODE_DIR)
config['CONFLUENCE_DATA_DIR'] = str(CONFLUENCE_DATA_DIR)

# Initialize logger
logging_manager = LoggingManager(config)
logger = logging_manager.logger

# Display key configuration settings
print("=== Directory Configuration ===")
print(f"Code Directory: {CONFLUENCE_CODE_DIR}")
print(f"Data Directory: {CONFLUENCE_DATA_DIR}")
print("\n=== Key Configuration Settings ===")
print(f"Domain Name: {config['DOMAIN_NAME']}")
print(f"Pour Point: {config['POUR_POINT_COORDS']}")
print(f"Spatial Mode: {config['SPATIAL_MODE']}")
print(f"Model: {config['HYDROLOGICAL_MODEL']}")
print(f"Simulation Period: {config['EXPERIMENT_TIME_START']} to {config['EXPERIMENT_TIME_END']}")

## 3. Step 1: Project Setup - Organizing the Modeling Workflow
The first step in any CONFLUENCE workflow is to establish a well-organized project structure. This might seem trivial, but it's crucial for:

Maintaining consistency across different experiments
Ensuring all components can find required files
Enabling reproducibility
Facilitating collaboration

In [ ]:
# Initialize the project manager
project_manager = ProjectManager(config, logger)

# Setup project directories
print("Creating project directory structure...")
project_dir = project_manager.setup_project()

# List created directories
print("\nCreated directories:")
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        print(f"  📁 {item.name}")

# Explain what each directory is for
print("\nDirectory purposes:")
print("  📁 shapefiles: Domain geometry (watershed, pour points, river network)")
print("  📁 attributes: Static characteristics (elevation, soil, land cover)")
print("  📁 forcing: Meteorological inputs (precipitation, temperature)")
print("  📁 simulations: Model outputs")
print("  📁 evaluation: Performance metrics and comparisons")
print("  📁 plots: Visualizations")
print("  📁 optimisation: Calibration results")

## 4. Step 2: Define the Pour Point - The Starting Point of Watershed Modeling
For lumped catchment modeling, everything begins with defining a single point: the watershed outlet or "pour point". This is where all water from the upstream area flows through. In our case, it's at the WSC gauging station where the Bow River flows past the town of Banff.

In [ ]:
# Create pour point shapefile from coordinates
print(f"Creating pour point shapefile from coordinates: {config['POUR_POINT_COORDS']}")
pour_point_path = project_manager.create_pour_point()

# Visualize the pour point
if pour_point_path and pour_point_path.exists():
    gdf = gpd.read_file(pour_point_path)
    
    # Reproject to Web Mercator for basemap compatibility
    gdf_web = gdf.to_crs(epsg=3857)
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Plot the pour point
    gdf_web.plot(ax=ax, color='red', markersize=200, marker='o', 
                 edgecolor='white', linewidth=2, zorder=5)
    
    # Add basemap
    cx.add_basemap(ax, 
                   source=cx.providers.CartoDB.Positron,
                   zoom=10,
                   alpha=0.8)
    
    # Calculate bounds with some padding
    minx, miny, maxx, maxy = gdf_web.total_bounds
    pad = 5000  # 5km padding
    ax.set_xlim(minx - pad, maxx + pad)
    ax.set_ylim(miny - pad, maxy + pad)
    
    # Add context
    lat, lon = gdf.geometry.iloc[0].y, gdf.geometry.iloc[0].x
    ax.text(minx + 1000, maxy - 1000,
            'Pour Point: Where all upstream\nwater flows through', 
            fontsize=14, 
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.8),
            fontweight='bold')
    
    ax.set_title(f'Pour Point Location: Bow River at Banff\nCoordinates: {lat:.4f}°N, {lon:.4f}°W', 
                fontsize=16, fontweight='bold', pad=20)
    
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    
    plt.tight_layout()
    plt.show()

## 5. Step 3: Acquire Geospatial Data - Characterizing the Landscape
Before we can delineate the watershed, we need elevation data. CONFLUENCE also acquires soil and land cover data at this stage for later use in the model.

In [ ]:
# Initialize data manager
from utils.geospatial.geospatial_utils import GeospatialDataAcquisition
geospatial_acquisition = GeospatialDataAcquisition(config, logger)

print("Acquiring geospatial attributes (DEM, soil, land cover)...")
print("This step downloads data from configured sources.")
print(f"Bounding box: {config['BOUNDING_BOX_COORDS']}")

# Acquire digital elevation model (DEM)
geospatial_acquisition.acquire_dem()

# Check downloaded files
attribute_dirs = {
    'DEM': project_dir / 'attributes' / 'elevation' / 'dem',
}

print("\nDownloaded attribute files:")
for name, path in attribute_dirs.items():
    if path.exists():
        files = list(path.glob('*.tif')) + list(path.glob('*.tiff'))
        print(f"  {name}: {len(files)} files")
        
print("\nThe DEM is crucial for:")
print("  - Watershed delineation")
print("  - Flow direction calculation")
print("  - Elevation-dependent processes")

## 6. Step 4: Watershed Delineation - Defining the Model Domain
Now comes a critical step: delineating the watershed boundary. This defines exactly which area contributes water to our pour point. CONFLUENCE uses terrain analysis algorithms to trace upstream from the pour point, identifying all areas that drain to it.

In [ ]:
# Initialize domain manager
from utils.geospatial.domain_manager import DomainManager
domain_manager = DomainManager(config, logger)

print(f"Delineating watershed using method: {config['DOMAIN_DEFINITION_METHOD']}")
print(f"Tool: {config['LUMPED_WATERSHED_METHOD']}")
print("\nWhat's happening:")
print("  1. Calculate flow directions from DEM")
print("  2. Trace upstream from pour point")
print("  3. Identify contributing area")
print("  4. Create watershed boundary")

# Delineate the domain
watershed_path = domain_manager.define_domain()

# Check outputs
basin_path = project_dir / 'shapefiles' / 'river_basins'
if basin_path.exists():
    basin_files = list(basin_path.glob('*.shp'))
    print(f"\nCreated {len(basin_files)} basin shapefile(s)")
    for f in basin_files:
        print(f"  - {f.name}")

## 7. Visualize the Delineated Domain
Let's see what our watershed looks like:

In [ ]:
# Visualize the watershed
fig, ax = plt.subplots(figsize=(12, 10))

# Load and plot watershed
if basin_files:
    basin_gdf = gpd.read_file(basin_files[0])
    
    # Reproject for visualization
    basin_web = basin_gdf.to_crs(epsg=3857)
    pour_web = gdf.to_crs(epsg=3857)
    
    # Plot watershed
    basin_web.plot(ax=ax, facecolor='lightblue', edgecolor='navy', 
                   linewidth=2, alpha=0.7)
    
    # Add pour point
    pour_web.plot(ax=ax, color='red', markersize=200, marker='o', 
                  edgecolor='white', linewidth=2, zorder=5)
    
    # Add basemap
    cx.add_basemap(ax, source=cx.providers.OSM.TopoMap, zoom=9)
    
    # Set extent
    minx, miny, maxx, maxy = basin_web.total_bounds
    pad = 5000
    ax.set_xlim(minx - pad, maxx + pad)
    ax.set_ylim(miny - pad, maxy + pad)
    
    # Add labels
    ax.text(minx + 1000, maxy - 1000,
            f'Watershed Area: {basin_gdf.geometry.area.sum() / 1e6:.0f} km²', 
            fontsize=14, 
            bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8),
            fontweight='bold')
    
    ax.set_title('Bow River Watershed at Banff\nAll water from this area flows to the pour point', 
                fontsize=16, fontweight='bold', pad=20)
    
    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.axis('off')
    
    plt.tight_layout()
    plt.show()

## 8. Step 5: Create Hydrologic Response Units (HRUs)
For a lumped model, the entire watershed becomes a single Hydrologic Response Unit (HRU). This simplification assumes uniform characteristics across the watershed - obviously an approximation, but useful for many applications.


In [ ]:
print(f"Creating lumped HRU using method: {config['DOMAIN_DISCRETIZATION']}")
print("\nFor lumped modeling:")
print("  - Entire watershed = 1 HRU")
print("  - Spatially averaged properties")
print("  - Simplified computational domain")

# Create HRU
hru_path = domain_manager.discretize_domain()

# Check the created HRU shapefile
hru_dir = project_dir / 'shapefiles' / 'catchment'
if hru_dir.exists():
    hru_files = list(hru_dir.glob('*.shp'))
    print(f"\nCreated {len(hru_files)} HRU shapefile(s)")
    
    if hru_files:
        # Load and display HRU properties
        hru_gdf = gpd.read_file(hru_files[0])
        print("\nHRU Properties:")
        print(f"Number of HRUs: {len(hru_gdf)}")
        print(f"Total area: {hru_gdf.geometry.area.sum() / 1e6:.2f} km²")
        
        # For lumped model, should be single HRU
        if len(hru_gdf) == 1:
            print("✓ Successfully created single lumped HRU")

## 9. Step 6: Process Observed Streamflow Data
To evaluate our model, we need observed streamflow data. CONFLUENCE can work with various data sources. For the Bow River, we'll use Water Survey of Canada (WSC) data.

In [ ]:
# Initialize data manager
from utils.data.data_utils import ObservedDataProcessor
obs_processor = ObservedDataProcessor(config, logger)

print(f"Processing observed streamflow data from: {config['STREAMFLOW_DATA_PROVIDER']}")
print(f"Station ID: {config['STATION_ID']}")
print("\nThis step:")
print("  - Downloads or loads streamflow observations")
print("  - Converts units if needed")
print("  - Resamples to model timestep")
print("  - Fills gaps where possible")

# Process streamflow data
obs_processor.process_streamflow_data()

# Visualize the data
obs_path = project_dir / 'observations' / 'streamflow' / 'preprocessed' / f"{config['DOMAIN_NAME']}_streamflow_processed.csv"
if obs_path.exists():
    obs_df = pd.read_csv(obs_path)
    obs_df['datetime'] = pd.to_datetime(obs_df['datetime'])
    
    print(f"\nProcessed streamflow data:")
    print(f"Period: {obs_df['datetime'].min()} to {obs_df['datetime'].max()}")
    print(f"Number of records: {len(obs_df)}")
    
    # Create informative plot
    fig, ax = plt.subplots(figsize=(14, 6))
    ax.plot(obs_df['datetime'], obs_df['discharge_cms'], 
            linewidth=1.5, color='blue', alpha=0.7)
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Discharge (m³/s)', fontsize=12)
    ax.set_title(f'Observed Streamflow - Bow River at Banff (WSC Station: {config["STATION_ID"]})\nShowing seasonal patterns and interannual variability', 
                fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    # Add context
    ax.text(0.02, 0.95, f'Mean: {obs_df["discharge_cms"].mean():.1f} m³/s\nMax: {obs_df["discharge_cms"].max():.1f} m³/s', 
            transform=ax.transAxes, 
            bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8),
            verticalalignment='top')
    
    plt.tight_layout()
    plt.show()

## 10. Step 7: Acquire Meteorological Forcing Data
Hydrological models need meteorological inputs (forcings) like precipitation and temperature. CONFLUENCE handles the complex process of downloading, processing, and formatting this data.
python

In [ ]:
# Initialize forcing data acquisition
from utils.data.data_utils import DataAcquisitionProcessor
data_processor = DataAcquisitionProcessor(config, logger)

print(f"Acquiring forcing data: {config['FORCING_DATASET']}")
print(f"Period: {config['EXPERIMENT_TIME_START']} to {config['EXPERIMENT_TIME_END']}")
print("\nThis step:")
print("  - Downloads gridded meteorological data")
print("  - Extracts data for our watershed")
print("  - Converts units to model requirements")
print("  - Handles missing data")

# Acquire forcing data
data_processor.run_data_acquisition()

# Check progress
forcing_path = project_dir / 'forcing' / 'easymore-outputs'
if forcing_path.exists():
    files = list(forcing_path.glob('*.nc'))
    print(f"\nProcessed {len(files)} forcing files")
    
    if files:
        # Quick look at the data
        import xarray as xr
        ds = xr.open_dataset(files[0])
        print(f"\nForcing variables: {list(ds.variables)}")
        print(f"Time period: {ds.time.values[0]} to {ds.time.values[-1]}")

## 11. Step 8: Model-Agnostic Preprocessing
Before running any specific hydrological model, CONFLUENCE performs model-agnostic preprocessing. This creates standardized inputs that can be used by different models.

In [ ]:
# Initialize model manager
from utils.models.model_manager import ModelManager
model_manager = ModelManager(config, logger)

print("Running model-agnostic preprocessing...")
print("This step:")
print("  - Calculates basin-averaged forcing")
print("  - Computes derived variables (like specific humidity)")
print("  - Creates model-agnostic NetCDF files")

# Run preprocessing
model_manager.prepare_inputs()

# Check outputs
basin_forcing_path = project_dir / 'forcing' / 'basin_averaged_data'
if basin_forcing_path.exists():
    files = list(basin_forcing_path.glob('*.nc'))
    print(f"\nCreated {len(files)} basin-averaged forcing files")
    
    # Show a sample of the processed data
    if files:
        ds = xr.open_dataset(files[0])
        print(f"\nProcessed variables: {list(ds.variables)}")

## 12. Step 9: Model-Specific Preprocessing
Now we prepare inputs specific to our chosen hydrological model (SUMMA in this case). Each model has its own requirements for input format and configuration.

In [ ]:
print(f"Preparing {config['HYDROLOGICAL_MODEL']} input files...")
print("This includes:")
print("  - Model configuration files")
print("  - Parameter files")
print("  - Initial conditions")
print("  - Output specifications")

# Run model-specific preprocessing
model_manager.configure_model()

# Check created files
settings_path = project_dir / 'settings' / config['HYDROLOGICAL_MODEL']
if settings_path.exists():
    files = list(settings_path.glob('*'))
    print(f"\nCreated {len(files)} model configuration files:")
    for f in files[:10]:  # Show first 10
        print(f"  - {f.name}")

## 13. Step 10: Run the Hydrological Model
Finally, we run the actual hydrological simulation. CONFLUENCE manages the execution and monitors progress.

In [ ]:
print(f"Running {config['HYDROLOGICAL_MODEL']} model...")
print("This may take several minutes depending on:")
print("  - Simulation period length")
print("  - Model complexity")
print("  - System performance")

# Run the model
model_manager.run_model()

# Check outputs
sim_path = project_dir / 'simulations' / config['EXPERIMENT_ID'] / config['HYDROLOGICAL_MODEL']
if sim_path.exists():
    files = list(sim_path.glob('*.nc'))
    print(f"\nModel completed. Created {len(files)} output files.")
    
    # Also check routing outputs if applicable
    if config.get('ROUTING_MODEL') == 'mizuRoute':
        mizu_path = project_dir / 'simulations' / config['EXPERIMENT_ID'] / 'mizuRoute'
        if mizu_path.exists():
            files = list(mizu_path.glob('*.nc'))
            print(f"Routing completed. Created {len(files)} output files.")

## 14. Step 11: Visualize and Evaluate Results
Let's see how our model performed compared to observations:

In [ ]:
# Initialize analysis manager
from utils.evaluation.analysis_manager import AnalysisManager
analysis_manager = AnalysisManager(config, logger)

print("Creating model evaluation plots...")
analysis_manager.evaluate_model()

# Load and plot results
results_path = project_dir / 'evaluation' / 'model_performance.csv'
if results_path.exists():
    results = pd.read_csv(results_path, index_col=0, parse_dates=True)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), height_ratios=[3, 1])
    
    # Time series comparison
    ax1.plot(results.index, results['observed'], label='Observed', 
             color='blue', linewidth=2, alpha=0.7)
    ax1.plot(results.index, results['simulated'], label='Simulated', 
             color='red', linewidth=1.5, alpha=0.7)
    
    ax1.set_ylabel('Discharge (m³/s)', fontsize=12)
    ax1.set_title(f'Model Results: {config["DOMAIN_NAME"]}\nComparing simulated vs observed streamflow', 
                  fontsize=14, fontweight='bold')
    ax1.legend(loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # Scatter plot
    ax2.scatter(results['observed'], results['simulated'], 
                alpha=0.5, s=20, color='purple')
    ax2.plot([0, results[['observed', 'simulated']].max().max()], 
             [0, results[['observed', 'simulated']].max().max()], 
             'k--', alpha=0.5, label='1:1 line')
    
    ax2.set_xlabel('Observed Discharge (m³/s)', fontsize=12)
    ax2.set_ylabel('Simulated Discharge (m³/s)', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Add performance metrics
    from utils.evaluation.metrics import calculate_metrics
    metrics = calculate_metrics(results['observed'], results['simulated'])
    
    metric_text = f"NSE: {metrics['NSE']:.3f}\nKGE: {metrics['KGE']:.3f}\nRMSE: {metrics['RMSE']:.2f} m³/s"
    ax1.text(0.02, 0.15, metric_text, transform=ax1.transAxes,
             bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8),
             verticalalignment='top', fontsize=11)
    
    plt.tight_layout()
    plt.show()

## 15. Summary: Understanding the CONFLUENCE Workflow
Congratulations! You've completed a full lumped basin modeling workflow with CONFLUENCE. Let's reflect on what we accomplished and how CONFLUENCE helped navigate the complex decision tree of hydrological modeling.
The Decision Tree We Navigated:

### Project Organization: Established a consistent structure for all files
Domain Definition: From pour point → watershed boundary → single HRU
Data Acquisition: Gathered forcing data, observations, and static attributes
Model Configuration: Set up SUMMA with appropriate parameters
Simulation: Ran the model for our specified period
Evaluation: Compared results with observations

## How CONFLUENCE Helped:

### Standardized each step into a clear, reproducible process
Handled complex data transformations behind the scenes
Maintained consistency across different data sources and formats
Provided sensible defaults while allowing customization
Created a complete record of all decisions and processes

## Next Steps You Could Try:

### Experiment with different models (change HYDROLOGICAL_MODEL)
Try distributed modeling (change SPATIAL_MODE to 'Distributed')
Calibrate the model (use the optimization module)
Analyze model sensitivity to different parameters
Compare multiple model structures (decision analysis)

## Key Outputs Created:

In [ ]:
# Print summary of key outputs
print("=== Workflow Complete ===\n")
print(f"Project: {confluence.config['DOMAIN_NAME']}")
print(f"Experiment: {confluence.config['EXPERIMENT_ID']}")
print(f"Model: {confluence.config['HYDROLOGICAL_MODEL']}")
print("=== Workflow Complete ===\n")
print(f"Project: {config['DOMAIN_NAME']}")
print(f"Experiment: {config['EXPERIMENT_ID']}")
print(f"Model: {config['HYDROLOGICAL_MODEL']}")
print(f"\nKey outputs:")
print(f"  - Watershed boundary: {project_dir}/shapefiles/river_basins/")
print(f"  - Model configuration: {project_dir}/settings/{config['HYDROLOGICAL_MODEL']}/")
print(f"  - Simulation results: {project_dir}/simulations/{config['EXPERIMENT_ID']}/")
print(f"  - Performance metrics: {project_dir}/evaluation/")
print(f"  - Visualizations: {project_dir}/plots/")

# Create a summary figure showing the workflow
fig, ax = plt.subplots(figsize=(12, 8))
ax.text(0.5, 0.5, 'CONFLUENCE Workflow Complete!', 
        ha='center', va='center', fontsize=24, fontweight='bold')
ax.text(0.5, 0.4, f'Successfully modeled {config["DOMAIN_NAME"]}', 
        ha='center', va='center', fontsize=16)
ax.text(0.5, 0.3, f'From {config["EXPERIMENT_TIME_START"]} to {config["EXPERIMENT_TIME_END"]}', 
        ha='center', va='center', fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()print(f"  - Watershed shapefile: shapefiles/river_basins/")
print(f"  - Model results: simulations/{confluence.config['EXPERIMENT_ID']}/")
print(f"  - Plots: plots/results/")
print(f"  - Forcing data: forcing/basin_averaged_data/")